# <center> Data Science - Laboratorio 6 -Analítica de Redes Sociales

* Daniela Villamar (19086)  
* Diego Crespo (19541)  
  

In [2]:
#Importamos las librerias a utilizar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neattext as nt
import neattext.functions as nfx
import unidecode
from neattext import TextMetrics
import re
from collections import Counter

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

Previo a cargar los datos, es importante especificar los mecanismos para la recolección. Utilizamos una página web que se encarga de recolectar los tweets en los últimos 10 días, a partir de una palabra clave o una frase. Dicho mecanismo fue escogido sobre la utilización de un API de Twitter puesto que era más rápido y permitía la reproducibilidad del trabajo. Cada palabra(s) clave generó, por su cuenta, un archivo .csv con la información pertinente respecto a los tweets, que luego unificamos en uno solo.

In [4]:
#Cargamos la data a utilizar en el laboratorio
data = pd.read_csv("data_tweets.csv", sep = ";")
data

,Tweet Id,Text,Name,Screen Name,UTC,Created At,Favorites,Retweets,Language,Client,Tweet Type,URLs,Hashtags,Mentions,Media Type
0,1435054999852175365,"Supuestamente iban a ser 3,000 sólo en el Parq...",•𝐉𝐏•,JPGramajoC,2021-09-07T01:39:08.000Z,Tue Sep 07 01:39:08 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/iphone"" r...",Tweet,https://twitter.com/evelynmorataya/status/1435...,2,1,NaN
1,1435054877579882499,RT @MinSaludGuate : Con la finalidad de apoyar...,Elizabeth Aguilar,mbettyaguilar,2021-09-07T01:38:39.000Z,Tue Sep 07 01:38:39 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,NaN,0,0,photo
2,1435054791768645633,RT @PrensaComunitar : Once médicos cubanos sum...,Judith Colón J.,JudithColnJ1,2021-09-07T01:38:19.000Z,Tue Sep 07 01:38:19 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,https://www.prensacomunitaria.org/2021/09/once...,0,0,NaN
3,1435054705558835202,RT @cronicagt : Un grupo de médicos del Hospit...,Jhony Cam,jhony_cam,2021-09-07T01:37:58.000Z,Tue Sep 07 01:37:58 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,https://cronica.com.gt/medicos-piden-a-diputad...,0,0,NaN
4,1435054053638168578,RT @lahoragt : Los directores de los hospitale...,Tamara Reyes,TamaraR77259349,2021-09-07T01:35:23.000Z,Tue Sep 07 01:35:23 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,NaN,0,0,photo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14225,1434655109754019840,RT @ManfredoGuate : Ante la posibilidad que el...,Daniel Perez,perez1k,2021-09-05T23:10:07.000Z,Sun Sep 05 23:10:07 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,NaN,1,0,photo
14226,1434654873824505856,RT @Tio_Politico : Quien diría que @msemillagt...,MariaM,MariaMartaU,2021-09-05T23:09:11.000Z,Sun Sep 05 23:09:11 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/iphone"" r...",Retweet,NaN,0,1,NaN
14227,1434654765565321219,RT @MVillacortaOf : MUCHO CIRCO Y POCO PAN:\r\...,Edgar Godinez,GodinezEdgar,2021-09-05T23:08:45.000Z,Sun Sep 05 23:08:45 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/iphone"" r...",Retweet,NaN,0,0,NaN
14228,1434654646413500420,RT @Lord_VaderTG : El novio del diputado semil...,tío ton,tioton31,2021-09-05T23:08:16.000Z,Sun Sep 05 23:08:16 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Retweet,NaN,1,0,photo


Tenemos la información de 14,230 tweets en español recientes. Algunos han sido retweteados, por ello debemos encarganos de limpiar la data para poder analizar, principalmente la información de los tweets

In [6]:
#Vemos la descripción de la data.
data.describe()

,Tweet Id,Favorites,Retweets,Hashtags,Mentions
count,1.423000e+04,14230.000000,14230.000000,14230.000000,14230.000000
mean,1.434597e+18,2.288897,0.615039,1.074209,0.494519
std,5.168183e+14,26.066325,6.627002,1.564452,1.101404
min,1.432012e+18,0.000000,0.000000,0.000000,0.000000
25%,1.434348e+18,0.000000,0.000000,0.000000,0.000000
50%,1.434727e+18,0.000000,0.000000,1.000000,0.000000
75%,1.435016e+18,0.000000,0.000000,1.000000,1.000000
max,1.435055e+18,1144.000000,287.000000,17.000000,22.000000


In [7]:
#Vemos que variables tienen datos nulos. 
data.isna().sum().to_frame()

,0
Tweet Id,0
Text,0
Name,0
Screen Name,0
UTC,0
Created At,0
Favorites,0
Retweets,0
Language,0
Client,0


Podemos observar que casi no hay datos faltantes en la mayoría de variables. Por otro lado, las variables que tienen datos faltantes no son tan relevantes para el estudio que se realizará por lo que podemos removerlas tranquilamente. En realidad, aquellos con elementos faltantes son pprque no incluyen dentro del texto una IRL o algún tipo de multimedia, elementos que no serán tomados en cuenta para el estudio.

In [8]:
# vemos las columnas de los datos
data.columns

Index(['Tweet Id', 'Text', 'Name', 'Screen Name', 'UTC', 'Created At',
       'Favorites', 'Retweets', 'Language', 'Client', 'Tweet Type', 'URLs',
       'Hashtags', 'Mentions', 'Media Type'],
      dtype='object')

#### Definimos un DataFrame para hacer la limpieza

In [9]:
# eliminamos las columnas que no vamos a utilizar
data_limpieza = data.drop(columns = ['Screen Name', 'UTC', 'Created At',
       'Favorites', 'Language', 'Client', 'URLs', 'Mentions', 'Media Type']) 

#### Removemos todas las variables que no son de interés o relevantes para el estudio

In [10]:
# observamos el DF resultante
data_limpieza

,Tweet Id,Text,Name,Retweets,Tweet Type,Hashtags
0,1435054999852175365,"Supuestamente iban a ser 3,000 sólo en el Parq...",•𝐉𝐏•,0,Tweet,2
1,1435054877579882499,RT @MinSaludGuate : Con la finalidad de apoyar...,Elizabeth Aguilar,0,Retweet,0
2,1435054791768645633,RT @PrensaComunitar : Once médicos cubanos sum...,Judith Colón J.,0,Retweet,0
3,1435054705558835202,RT @cronicagt : Un grupo de médicos del Hospit...,Jhony Cam,0,Retweet,0
4,1435054053638168578,RT @lahoragt : Los directores de los hospitale...,Tamara Reyes,0,Retweet,0
...,...,...,...,...,...,...
14225,1434655109754019840,RT @ManfredoGuate : Ante la posibilidad que el...,Daniel Perez,0,Retweet,1
14226,1434654873824505856,RT @Tio_Politico : Quien diría que @msemillagt...,MariaM,0,Retweet,0
14227,1434654765565321219,RT @MVillacortaOf : MUCHO CIRCO Y POCO PAN:\r\...,Edgar Godinez,0,Retweet,0
14228,1434654646413500420,RT @Lord_VaderTG : El novio del diputado semil...,tío ton,0,Retweet,1


## Limpieza del texto

In [ ]:
# definimos la columna a ser limpiada
s = data_limpieza["Text"]

#### Pasamos todo a mayúsuclas

In [12]:
s = s.str.upper()

#### Removemos los Stopwords (en Español)

In [13]:
for i in range(len(s)):
    s[i] = nt.TextFrame(str(s[i])).remove_stopwords(lang = "es")

In [14]:
# observamos a simple vista los resultados
s

0        SUPUESTAMENTE IBAN A 3,000 PARQUE INDUSTRIA, ¿...
1        RT @MINSALUDGUATE : FINALIDAD APOYAR A PAÍS PA...
2        RT @PRENSACOMUNITAR : ONCE MÉDICOS CUBANOS SUM...
3        RT @CRONICAGT : GRUPO MÉDICOS HOSPITAL TEMPORA...
4        RT @LAHORAGT : DIRECTORES HOSPITALES, ROOSEVEL...
                               ...                        
14225    RT @MANFREDOGUATE : POSIBILIDAD CONGRESO APRUE...
14226    RT @TIO_POLITICO : DIRÍA @MSEMILLAGT ALIADE PR...
14227    RT @MVILLACORTAOF : CIRCO Y PAN: PROFUNDA TRIS...
14228    RT @LORD_VADERTG : NOVIO DIPUTADO SEMILLERO RO...
14229    RT @REALRODRIGOPOLO : PARDO PINEDA APOYANDO RO...
Name: Text, Length: 14230, dtype: object

#### Removemos E - Mails, Números, Números de Teléfono, BTC Adresses, URL's, Emojis, HTML Tags

In [15]:
for i in range(len(s)):
    s[i] = nt.TextFrame(str(s[i])).remove_emails()
    s[i] = nt.TextFrame(str(s[i])).remove_numbers()
    s[i] = nt.TextFrame(str(s[i])).remove_phone_numbers()
    s[i] = nt.TextFrame(str(s[i])).remove_btc_address()
    s[i] = nt.TextFrame(str(s[i])).remove_urls()
    s[i] = nt.TextFrame(str(s[i])).remove_emojis()
    s[i] = nt.TextFrame(str(s[i])).remove_html_tags()
    s[i] = nt.TextFrame(str(s[i])).remove_stopwords(lang = "es")

#### Removemos las tildes

In [16]:
for i in range(len(s)):
    s[i] = unidecode.unidecode(str(s[i]))
    s[i] = re.sub(' +', ' ',str(s[i]))
    s[i] = nt.TextFrame(str(s[i])).remove_stopwords(lang = "es")

#### Removemos signos de puntuación

In [17]:
for i in range(len(s)):
    s[i] = nt.TextFrame(str(s[i])).remove_puncts()

In [18]:
# vemos los resultados obtenidos luego de los pasos anteriores
s

0        SUPUESTAMENTE IBAN A  PARQUE INDUSTRIA VAA VOS...
1        RT @MINSALUDGUATE : FINALIDAD APOYAR A PANDEMI...
2        RT @PRENSACOMUNITAR : ONCE MEDICOS CUBANOS SUM...
3        RT @CRONICAGT : GRUPO MEDICOS HOSPITAL TEMPORA...
4        RT @LAHORAGT : DIRECTORES HOSPITALES ROOSEVELT...
                               ...                        
14225    RT @MANFREDOGUATE : POSIBILIDAD CONGRESO APRUE...
14226    RT @TIOPOLITICO : DIRIA @MSEMILLAGT ALIADE PRE...
14227    RT @MVILLACORTAOF : CIRCO Y PAN: PROFUNDA TRIS...
14228    RT @LORDVADERTG : NOVIO DIPUTADO SEMILLERO ROM...
14229    RT @REALRODRIGOPOLO : PARDO PINEDA APOYANDO RO...
Name: Text, Length: 14230, dtype: object